# Model class

In [2]:
import torch
import torch.nn as nn

In [11]:
class StocksPredictionModel(nn.Module):
    '''
        Let's break all the parameters down:
            num_classes - how many features will our model have
            hidden size - how many features in the hidden state 
            num_layers - how deep our model is
    '''
    def __init__(self, num_classes, input_size, hidden_size, num_layers, batch_first = True):
        super().__init__()
       
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.custom_ltsm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, batch_first = batch_first)
        self.linear_sequence = nn.Sequential(
            nn.ReLU(),
            nn.Linear(hidden_size, 256, bias=True),
            nn.ReLU(),
            nn.Linear(256, 128, bias=True),
            nn.ReLU(),
            nn.Linear(128, 64, bias=True),
            nn.ReLU(),
            nn.Linear(64, num_classes))

    # Basically forward function is what we should do with data
    def forward(self, x):
       
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()

        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()

        output, (hn, cn) = self.custom_ltsm(x, (h0, c0))
        hn = hn.view(-1, self.hidden_size)
       
        return self.linear_sequence(hn)